## Notebook to prepare the data from nnUnet segmentations and MSD dataset

Optional step, if you require it. 


**NOTE:** run this before running `data_preparation.py`

In [ ]:
%cd ..

In [ ]:
import nibabel as nib
import numpy as np
import os

In [ ]:
#rand_id = np.random.randint(0, len(os.listdir('nnUnet_seg/hippocampus/Ts')))
#print(rand_id)
rand_img = nib.load(f'/data/marciano/experiments/multi-organ-qc/MSD_data/Task02_Heart/labelsTr/la_003.nii.gz').get_fdata().transpose(2, 0, 1)
rand_img.shape

In [ ]:
rand_img = nib.load(f'/data/marciano/experiments/multi-organ-qc/data/prostate/structured/patient013/mask.nii.gz').get_fdata().transpose(2, 0, 1)
rand_img.shape

In [ ]:
np.unique(rand_img)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(rand_img.squeeze())
plt.show()

In [ ]:
for i,folder in enumerate(os.listdir("/data/marciano/experiments/multi-organ-qc/data/prostate/reconstructions/")):
    rand_img = nib.load('/data/marciano/experiments/multi-organ-qc/data/prostate/structured/patient{:03d}/mask.nii.gz'.format(i)).get_fdata().squeeze().argmax(0)
    if len(np.unique(rand_img).shape)>2: print(i, np.unique(rand_img))

In [ ]:
from utils.dataset import select_valid_imgs

label_path = '/data/marciano/experiments/multi-organ-qc/MSD_data/Task05_Prostate/labelsTr'
save_path_label = '/data/marciano/experiments/multi-organ-qc/data/prostate/labels'
select_valid_imgs(label_path, save_path_label, inter_slice_range=[3, 18], non_zero_thres=0.001)



# 3D interactive plot

In [ ]:
r, c = rand_img.shape[1:]

# Define frames
import numpy as np
from skimage import io
import plotly.graph_objects as go
nb_frames = rand_img.shape[0]

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(nb_frames*.1 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(rand_img[(nb_frames-1) - k]),
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=nb_frames*.1 * np.ones((r, c)),
    surfacecolor=np.flipud(rand_img[(nb_frames-1)]),
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                zaxis=dict(range=[-0.1, nb_frames*.1], autorange=False),
                aspectratio=dict(x=1, y=1, z=1),
                ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(100)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()